In [ ]:
# -*- coding: utf-8 -*-
import os
import cv2
from sklearn.cross_validation import train_test_split
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import numpy as np

In [ ]:
cd

In [ ]:
cd desktop/data/peach-data

In [ ]:
def photo():
    data_dir_path = u"./119APPLE/extended"
    file_list = os.listdir(r'./119APPLE/extended')
    image_data=[]
    label = []

    for file_name in tqdm(file_list):
        if file_name == ".DS_Store":
            continue
        if file_name == "extended":
            continue
        root, ext = os.path.splitext(file_name)
        if ext == u'.png' or u'.jpeg' or u'.jpg':
            abs_name = data_dir_path + '/' + file_name
            #print(file_name)
            image=cv2.imread(abs_name)
            image = cv2.resize(image, ( 227, 227 ))
            image = image.transpose(2, 0, 1)
            #以下各画像に対する処理を記載する
            
            image_data.append(image/255)
            label.append(0)

            
    
    return image_data,label


In [ ]:
def photo_1():
    data_dir_path = u"./120APPLE/extended_2"
    file_list = os.listdir(r'./120APPLE/extended_2')
    image_data=[]
    label=[]

    for file_name in tqdm(file_list):
        if file_name == ".DS_Store":
            continue
        if file_name == "extended_2":
            continue
        root, ext = os.path.splitext(file_name)
        if ext == u'.png' or u'.jpeg' or u'.jpg':
            abs_name = data_dir_path + '/' + file_name
            image = cv2.imread(abs_name)
            image = cv2.resize(image, ( 227, 227 ))
            image = image.transpose(2, 0, 1)
            
           
            image_data.append(image/255)
            label.append(1)

            
    
    return image_data,label


In [ ]:
data,label = photo()

In [ ]:
for i in tqdm(range(5)):
    plt.subplot(1, 5, i + 1)
    plt.imshow(data[i])
    
plt.show()

In [ ]:
data_1,label_1=photo_1()

In [ ]:
plt.imshow(data_1[1])
plt.show()

In [ ]:
data.extend(data_1)

In [ ]:
label.extend(label_1)

In [ ]:
np.shape(data)

In [ ]:
print(type(data))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Conv2D, Dense, Flatten, MaxPooling2D, BatchNormalization
from keras.models import Sequential, load_model
from keras.utils.np_utils import to_categorical
from keras.callbacks import Callback, EarlyStopping
from keras.optimizers import SGD
from keras.optimizers import Adam

In [ ]:
train_data, test_data, train_label, test_label = train_test_split(data, label, test_size=0.3, random_state=0)

In [ ]:
train_data = np.reshape(a=train_data, newshape=(-1,3,227,227))
test_data = np.reshape(a = test_data,newshape=(-1,3,227,227))
train_labels = to_categorical(train_label,2)
test_labels = to_categorical(test_label,2)

In [ ]:
print(np.shape(test_data))
print(np.shape(train_labels))

In [ ]:
from keras.layers import Activation, Dense, Dropout, Convolution2D, Flatten, MaxPooling2D,BatchNormalization
from keras import backend as K
K.set_image_dim_ordering('tf')
K.set_image_data_format('channels_first')
model2 = Sequential()

In [ ]:
#Alex net
#kernelの大きさは分割する際のピクセル数
model2.add(Conv2D(input_shape=(3,227, 227), filters=96,kernel_size=(7, 7), strides=(2, 2), padding="same"))
model2.add(MaxPooling2D(pool_size=(3, 3),strides=(2,2),border_mode='same'))
model2.add(BatchNormalization())

model2.add(Conv2D(filters=256, kernel_size=(
    5, 5), strides=(2, 2), padding="same",bias_initializer='ones' ))
model2.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2),border_mode='same'))
#model2.add(Activation("relu"))
model2.add(BatchNormalization())


model2.add(Conv2D(filters=384,kernel_size= (3,3),bias_initializer='zeros' ))
model2.add(Conv2D(filters=384, kernel_size= (3,3),bias_initializer='ones'))
model2.add(Conv2D(filters=256, kernel_size= (3,3),bias_initializer='ones',padding="same"))
model2.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2),border_mode='same'))
model2.add(BatchNormalization())



model2.add(Flatten())
model2.add(Dense(4096))
model2.add(Dropout(0.5))
model2.add(Dense(4096))
model2.add(Dropout(0.5))


model2.add(Dense(2))
model2.add(Activation('softmax'))

# コンパイル
model2.compile(optimizer=SGD(lr=0.0001), loss='categorical_crossentropy',
              metrics=['accuracy'])
#SGDの大きさは学習率s
# 学習
early_stopping = EarlyStopping(monitor='loss', patience=3, verbose=1, mode='auto')
history = model2.fit(train_data, train_labels, batch_size=128, epochs=18,shuffle=True,validation_data=(test_data, test_labels),callbacks=[early_stopping])


In [ ]:
def plot_history(history):
    #plt.plot(history.history['acc'],"o-",label="accuracy")
    plt.plot(history.history['val_acc'],"o-",label="val_acc")
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.ylim(0, 1)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()

    #plt.plot(history.history['loss'],"o-",label="loss",)
    plt.plot(history.history['val_loss'],"o-",label="val_loss")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.ylim(ymin=0)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()


In [ ]:
plot_history(history)

In [ ]:
model2.save("model_alex4.h5")

In [ ]:
model2.summary()

In [ ]:
model=load_model("model_alex3.h5")

In [ ]:
model.compile(optimizer=SGD(lr=0.0001), loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
score = model.evaluate(test_data, test_labels, verbose=0)
print('Test loss :', score[0])
print('Test accuracy :', score[1])

In [ ]:
from sklearn.metrics import confusion_matrix
data_pred = model.predict_classes(test_data,batch_size=1)

In [ ]:
confmat = confusion_matrix(test_label,y_pred=data_pred )

In [ ]:
print(confmat)

In [ ]:
fig,ax = plt.subplots(figsize=(2.5,2.5))
ax.matshow(confmat,cmap=plt.cm.Blues,alpha=0.3)
for i in range(confmat.shape[0]):
    for j in range(confmat.shape[1]):
        ax.text(x=j,y=i,s=confmat[i,j],va='center',ha='center')
plt.xlabel('predicted label')
plt.ylabel('true label')
plt.tight_layout()
plt.show

In [ ]:
sample=[]
for i in tqdm(range(100)):
    if test_label[i-1] != data_pred[i-1]:
        test=np.reshape(a =  test_data[i-1],newshape=(3,227,227))
        test = test.transpose(1, 2, 0)
        #tested = cv2.cvtColor(test, cv2.COLOR_BGR2RGB)
        #tested = cv2.cvtColor(test, cv2.COLOR_RGB2BGR)
        sample.append(test)
        
        

In [ ]:

for i in tqdm(range(5)):
    plt.subplot(1, 5, i + 1)
    plt.imshow(sample[i])
    
plt.show()

In [ ]:
#sam=cv2.cvtColor(sample[3], cv2.COLOR_BGR2RGB)
plt.figure(figsize=(50, 50))
sam=cv2.resize(sample[3], ( 227, 227 ))
plt.imshow(sam)
plt.show()

In [ ]:
plt.savefig('figure.png')

In [ ]:
from PIL import Image
 
categories = [ "0","1"]
 
pre = data_pred
for i,v in enumerate(pre):
    pre_ans = v.argmax()
    ans = test_labels[i].argmax()
    dat = test_data[i]
    if ans == pre_ans: continue
    fname = "NG_photo/" + str(i) + "-" + categories[pre_ans] + "-ne-" + categories[ans] + ".png"
    #dat *= 255
    img = Image.fromarray(dat.reshape((227,227,3))).convert("RGB")
    img.save(fname)